## Computer Vision Task 10

### Libraries Used

In [1]:
import numpy as np
import cv2

### Applying stereo vision to calculate and draw a disparity map.

In [ ]:
minDisparity = 16
numDisparities = 192 - minDisparity
blockSize = 5
p1 = 600
p2 = 2400

stereo = cv2.StereoSGBM_create(
    minDisparity=minDisparity,
    numDisparities=numDisparities,
    blockSize=blockSize,
    P1=p1,
    P2=p2
)

imgLeft = cv2.imread("left.png") # left image path
imgRight = cv2.imread("right.png") # right image path

# Check if images are loaded properly
if imgLeft is None or imgRight is None:
    print("Failed to load one or both of the images.")
    exit()

# Check if images have the same type
if imgLeft.dtype != imgRight.dtype:
    print("The left and right images have different types.")
    exit()

# Check if images are of type CV_8U (8-bit unsigned integers)
if imgLeft.dtype != np.uint8:
    print("The left and right images are not of type CV_8U.")
    exit()

# Resize images to have the same dimensions
imgLeft = cv2.resize(imgLeft, (imgRight.shape[1], imgRight.shape[0]))

def update(_):
    stereo.setBlockSize(cv2.getTrackbarPos("blockSize", "Disparity"))
    
    disparity = stereo.compute(imgLeft, imgRight).astype(np.float32) / 16.0
    
    cv2.imshow("Left", imgLeft)
    cv2.imshow("Right", imgRight)
    cv2.imshow("Disparity", disparity - minDisparity / numDisparities)

cv2.namedWindow("Disparity")
cv2.createTrackbar("blockSize", "Disparity", blockSize, 21, update)

update(0)  # Initial call to update

while True:
    key = cv2.waitKey(1)
    if key == 27:  # Exit on Esc key
        break

cv2.destroyAllWindows()

### Question1: write the python code to perform stereo vision on 2 images to compute the disparity without parameters update.

In [10]:
def compute_disparity(left_img, right_img, block_size=5, max_disparity=64):
    # Convert images to grayscale
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # Ensure both images have the same dimensions
    if left_gray.shape != right_gray.shape:
        raise ValueError("Left and right images must have the same dimensions.")

    # Initialize the disparity map
    disparity_map = np.zeros_like(left_gray, dtype=np.float32)

    # Iterate over each pixel in the left image
    for y in range(block_size, left_gray.shape[0] - block_size):
        for x in range(block_size, left_gray.shape[1] - block_size):
            best_offset = 0
            min_ssd = float('inf')

            # Iterate over a range of possible disparities
            for offset in range(max_disparity):
                left_patch = left_gray[y-block_size:y+block_size+1, x-block_size:x+block_size+1]
                right_patch = right_gray[y-block_size:y+block_size+1, x-block_size-offset:x+block_size-offset+1]

                if right_patch.shape[1] != left_patch.shape[1]:
                    continue

                ssd = np.sum((left_patch - right_patch) ** 2)
                
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_offset = offset

            # Assign the disparity value to the disparity map
            disparity_map[y, x] = best_offset

    return disparity_map

# Load the left and right images
left_image = cv2.imread('left.png') # left image path
right_image = cv2.imread('right.png') # right image path

# Resize the images to a common size
width = min(left_image.shape[1], right_image.shape[1])
height = min(left_image.shape[0], right_image.shape[0])
left_image = cv2.resize(left_image, (width, height))
right_image = cv2.resize(right_image, (width, height))

# Compute the disparity map
disparity_map = compute_disparity(left_image, right_image)

# Display the disparity map
cv2.imshow('Disparity Map', disparity_map / disparity_map.max())
cv2.waitKey(0)
cv2.destroyAllWindows()

### Question2: write the python code to perform stereo vision on 2 images to compute the disparity and allow blocksize parameter to be updated

In [11]:
def compute_disparity(left_img, right_img, block_size=5, max_disparity=64):
    # Convert images to grayscale
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # Ensure both images have the same dimensions
    if left_gray.shape != right_gray.shape:
        raise ValueError("Left and right images must have the same dimensions.")

    # Initialize the disparity map
    disparity_map = np.zeros_like(left_gray, dtype=np.float32)

    # Iterate over each pixel in the left image
    for y in range(block_size, left_gray.shape[0] - block_size):
        for x in range(block_size, left_gray.shape[1] - block_size):
            best_offset = 0
            min_ssd = float('inf')

            # Iterate over a range of possible disparities
            for offset in range(max_disparity):
                left_patch = left_gray[y-block_size:y+block_size+1, x-block_size:x+block_size+1]
                right_patch = right_gray[y-block_size:y+block_size+1, x-block_size-offset:x+block_size-offset+1]

                if right_patch.shape[1] != left_patch.shape[1]:
                    continue

                ssd = np.sum((left_patch - right_patch) ** 2)
                
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_offset = offset

            # Assign the disparity value to the disparity map
            disparity_map[y, x] = best_offset

    return disparity_map

# Load the left and right images
left_image = cv2.imread('left.png') # left image path
right_image = cv2.imread('right.png') # right image path

# Resize the images to a common size
width = min(left_image.shape[1], right_image.shape[1])
height = min(left_image.shape[0], right_image.shape[0])
left_image = cv2.resize(left_image, (width, height))
right_image = cv2.resize(right_image, (width, height))

# adjust the size of the block used for computing disparity.
# The larger the block size, the more information is considered in the matching process, but it can also lead to decreased accuracy in regions with depth discontinuities.
# Set the block size (you can modify this value)
block_size = 9

# Compute the disparity map
disparity_map = compute_disparity(left_image, right_image, block_size=block_size)

# Display the disparity map
cv2.imshow('Disparity Map', disparity_map / disparity_map.max())
cv2.waitKey(0)
cv2.destroyAllWindows()

### The End